In [0]:
%run ../01-Config/02-config

In [0]:
query = """
CREATE TABLE IF NOT EXISTS ECOMMERCE_DB.SILVER_DB.TRANSACTIONS (
    transaction_id STRING,
    customer_id STRING,
    product_id STRING,
    quantity INT,
    total_amount DOUBLE,
    transaction_date DATE,
    payment_method STRING,
    store_type STRING,
    order_status STRING,
    last_updated TIMESTAMP
)"""
spark._jvm.net.snowflake.spark.snowflake.Utils.runQuery(snowflake_config, query)

In [0]:
# Get the last processed timestamp from silver layer
customer_query = "SELECT MAX(last_updated) as last_processed FROM ECOMMERCE_DB.SILVER_DB.TRANSACTIONS"
last_processed_df = spark.read.format("snowflake") \
    .options(**snowflake_config) \
    .option("query", customer_query) \
    .load()
# Display the schema to check the column names
last_processed_df.printSchema()

# Check if the DataFrame is empty
if last_processed_df.count() == 0:
    last_processed_timestamp = "1900-01-01T00:00:00.000+00:00"
else:
    last_processed_row = last_processed_df.collect()[0]
    last_processed_timestamp = last_processed_row['LAST_PROCESSED'] if 'LAST_PROCESSED' in last_processed_row else None
    if last_processed_timestamp is None:
        last_processed_timestamp = "1900-01-01T00:00:00.000+00:00"

In [0]:
query = f"""
SELECT *
FROM ECOMMERCE_DB.BRONZE.transaction c 
WHERE c.ingestion_timestamp > '{last_processed_timestamp}'
"""
transaction_view = spark.read.format("snowflake") \
    .options(**snowflake_config) \
    .option("query", query) \
    .load() 

#customer_view.createOrReplaceTempView("bronze_incremental")


In [0]:
transaction_view.createOrReplaceTempView("transaction_incremental")
spark.sql("select * from transaction_incremental").show()

Data Transformations:
   - Quantity and total_amount normalization (setting negative values to 0)
   - Date casting to ensure consistent date format
   - Order status derivation based on quantity and total_amount

Data Quality Checks: We filter out records with null transaction dates, customer IDs, or product IDs.


In [0]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW silver_incremental_orders AS
SELECT
    transaction_id,
    customer_id,
    product_id,
    CASE 
        WHEN quantity < 0 THEN 0 
        ELSE quantity 
    END AS quantity,
    CASE 
        WHEN total_amount < 0 THEN 0 
        ELSE total_amount 
    END AS total_amount,
    CAST(transaction_date AS DATE) AS transaction_date,
    payment_method,
    store_type,
    CASE
        WHEN quantity = 0 OR total_amount = 0 THEN 'Cancelled'
        ELSE 'Completed'
    END AS order_status,
    CURRENT_TIMESTAMP() AS last_updated
FROM transaction_incremental
WHERE transaction_date IS NOT NULL
  AND customer_id IS NOT NULL
  AND product_id IS NOT NULL
""")

In [0]:
spark.sql("select * from silver_incremental_orders").show()

In [0]:
customer_query = "SELECT * FROM ECOMMERCE_DB.SILVER_DB.TRANSACTIONS"
target_df = spark.read.format("snowflake") \
    .options(**snowflake_config) \
    .option("query", customer_query) \
    .load()

display(target_df)

source_cust = spark.sql("select * from silver_incremental_orders")
upd_cust = target_df.join(source_cust, "transaction_id", "inner").select(source_cust["*"])
new_cust = target_df.join(source_cust, "transaction_id", "right").select(source_cust["*"])

display(upd_cust)
display(new_cust)




In [0]:
#wite data staging tables to silver schema in snowflake

new_cust.write \
    .format("snowflake") \
    .options(**snowflake_config) \
    .option("dbtable", 'transactions') \
    .option("sfDatabase", 'ecommerce_db') \
    .option("sfSchema", 'silver_db') \
    .mode("append") \
    .save()

upd_cust.write \
    .format("snowflake") \
    .options(**snowflake_config) \
    .option("dbtable", 'transactions') \
    .option("sfDatabase", 'ecommerce_db') \
    .option("sfSchema", 'silver_db') \
    .mode("overwrite") \
    .save()


